In [1]:
import json
import os
import pandas as pd
import csv
import keras
from numpy import array
import matplotlib.pyplot as plt
import subprocess

In [2]:
from google.colab import drive
drive.mount('/content/drive')
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive  sample_data


In [ ]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  
from IPython.display import YouTubeVideo

Selecting previously unselected package libgflags2.2.
(Reading database ... 160980 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.1-1_amd64.deb ...
Unpacking libgflags2.2 (2.2.1-1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.1-1_amd64.deb ...
Unpacking libgflags-dev (2.2.1-1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.3.5-1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.3.5-1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.3.5-1) ...
Selecting previously unselected package libhdf5-serial-dev.
Preparing to unpack .../04-libhdf5-serial-dev_1.10.0-patch1+docs-4_all.deb ...
Unpacking libhdf5-serial-dev (1.10.0-patch1+docs-4) ...
Selecting previously unselected package libleveldb1v5:amd64.
Preparing to unpack ...

In [ ]:
cd openpose

/content/openpose


In [ ]:
ls "/content/drive/MyDrive/Colab Notebooks/Youtube/output"

test_01_000000000000_keypoints.json  test_01_000000009145_keypoints.json
test_01_000000000001_keypoints.json  test_01_000000009146_keypoints.json
test_01_000000000002_keypoints.json  test_01_000000009147_keypoints.json
test_01_000000000003_keypoints.json  test_01_000000009148_keypoints.json
test_01_000000000004_keypoints.json  test_01_000000009149_keypoints.json
test_01_000000000005_keypoints.json  test_01_000000009150_keypoints.json
test_01_000000000006_keypoints.json  test_01_000000009151_keypoints.json
test_01_000000000007_keypoints.json  test_01_000000009152_keypoints.json
test_01_000000000008_keypoints.json  test_01_000000009153_keypoints.json
test_01_000000000009_keypoints.json  test_01_000000009154_keypoints.json
test_01_000000000010_keypoints.json  test_01_000000009155_keypoints.json
test_01_000000000011_keypoints.json  test_01_000000009156_keypoints.json
test_01_000000000012_keypoints.json  test_01_000000009157_keypoints.json
test_01_000000000013_keypoints.json  test_01_000000

In [ ]:
# now we create landmarks for these videos
video_path = "/content/drive/MyDrive/Colab Notebooks/Youtube/newVideo"
landmark_path = "/content/drive/MyDrive/Colab Notebooks/Youtube/output/"
for items in os.listdir(video_path):
  if len(items.split(".")) != 0:
    videoPath = os.path.join(video_path, items)
    command = './build/examples/openpose/openpose.bin --video "' + str(videoPath) +  '" --write_json "' + str(landmark_path) + '" --display 0 --render_pose 0 --net_resolution 1312x736 --scale_number 4 --scale_gap 0.25'
    x = subprocess.run(command, shell=True, capture_output=True)
    print(x.stdout.decode())

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 2147.965938 seconds.

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 2071.808512 seconds.

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 4355.842202 seconds.

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 1020.006123 seconds.

Starting OpenPose demo...
Configuring OpenPose...
Starting threa

Starting here, we process the landmark data created by openpose on each video

In [70]:
# this function reads in all the landmarks, filter out all the landmarks without detecting any person, and return the data as a list
def readData(path):
    frameCounter = []
    emptyFrame = []
    totalData = []
    dataFrame = []
    for itemPath in os.listdir(path):
        names = itemPath.split("_")
        if len(names) < 4:
          continue      
        fullPath = os.path.join(path, itemPath)    
        Data = pd.read_json(fullPath) 
        pData = Data["people"]
        poseData = []
        if len(pData) != 0:  
          fullData = pData[0]["pose_keypoints_2d"]                  
          poseData.append(fullData[:45] + fullData[57:])
          dataFrame.append(itemPath.split('.')[0])
          totalData+=poseData
        else:
          emptyFrame.append(itemPath.split('.')[0])                
    return totalData,dataFrame, emptyFrame

In [71]:
# we need to normalize data
def normalizeData(totalData, width, height):
    for data in totalData:
        for i in range(0, 63, 3):
            data[i] = float(data[i]) / width
            data[i+1] = float(data[i+1]) / height
    return totalData

In [72]:
# this segment create a list that contains already processed videos
video_path = "/content/drive/MyDrive/Colab Notebooks/Youtube/oldVideo"
landmark_path = "/content/drive/MyDrive/Colab Notebooks/Youtube/output/"
csv_path = "/content/drive/MyDrive/Colab Notebooks/Youtube/output/csvData"
existedData = []
for csvItem in os.listdir(csv_path):
  names = csvItem.split(".")
  if len(names) == 2:
    existedData.append(names[0])
print(existedData)


['test_10', 'test_08', 'test_09', 'test_03', 'test_07', 'test_05', 'test_04', 'test_06', 'test_02', 'test_01']


In [73]:
# this segment produces the csv file for each youtube video
for folderName in os.listdir(landmark_path):
  if len(folderName.split("_")) < 2:
    continue     
  if folderName in existedData:
    continue  
  folderPath = os.path.join(landmark_path, folderName)
  print(folderPath)
  totalData,dataFrame, emptyFrame = readData(folderPath)
  normalizedD = normalizeData(totalData, 1280, 720)
  df = pd.DataFrame.from_records(normalizedD)
  df.insert(63, "frameName", dataFrame)
  csvName = "/content/drive/MyDrive/Colab Notebooks/Youtube/output/" + str(folderName) + ".csv"
  print("saving csv for " + csvName)
  df.head()
  df.to_csv(csvName)

In [74]:
# now we do the prediction of the video
# function that returns the prediction of a video 
def predictData(videoName, basePath):
  model = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/projectModelBILSTM.h5")  
  path = os.path.join(basePath, videoName + ".csv")
  print("now processing " + path)
  df = pd.read_csv(path)
  df = df.iloc[:, 1:]
  frame = df.loc[:, 'frameName'].values
  df = df.drop(columns="frameName")
  x_test = df.values
  x_test = array(x_test).reshape((len(x_test), 1, len(x_test[0])))
  print("shape of x_test: ")
  print(x_test.shape)
  print("number of landmarks: ")
  print(len(x_test))
  probability = model.predict(x_test)
  return probability, frame


In [75]:
# this reads in the prediction by the model and produce the corresponding meta data
def createJson(probability, frameName, videoID):
  trueFlag = False
  data = []
  startTimeList = []
  endTimeList = []
  confidanceMax = 0
  confidanceList = []
  for index in range(len(probability)):
    if probability[index] > 0.95:      
      if trueFlag == False:
        confidanceMax = probability[index]
        trueFlag = True
        startTimeList.append(float(frameName[index].split("_")[2]) / 30) 
      if trueFlag == True:
        confidanceMax = max(probability[index], confidanceMax)
    else:
      if trueFlag == True:
        trueFlag = False
        confidanceList.append(confidanceMax)
        endTimeList.append(float(frameName[index].split("_")[2]) / 30)
  
  if trueFlag == True:
    confidanceList.append(confidanceMax)
    endTimeList.append(float(frameName[-1].split("_")[2])/ 30)

  if len(startTimeList) != len(endTimeList):
    print("SOMETHING IS WRONG!!!")
    return

  for index in range(len(startTimeList)):
    if startTimeList[index] > endTimeList[index]:
      continue
    if endTimeList[index] - startTimeList[index] < 0.2:
      continue
    newDict = {}
    newDict["videoId"] = str(vidoeID)
    newDict["type"] = "segment"
    newDict["startTime"] = float(startTimeList[index])
    newDict["endTime"] = float(endTimeList[index])
    newDict["observer"] = "CSCE636-Spring2021-MattDLSmartHome-v5"
    newDict["isHuman"] = False
    newDict["confirmedBySomeone"] = False
    newDict["rejectedBySomeone"] = False
    newDict["observation"] = {"label": "Fall", "labelConfidance": float(confidanceList[index])}     
    data.append(newDict)
  return data


  

In [76]:
# first read the video URL to extract the video ID
testURL = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Youtube/testURL.xlsx", header=0)
testInfo = {}
for index, row in testURL.iterrows():
  vidoeID = row[1].split('?')[1][2:]
  testInfo[row[0]] = vidoeID

In [77]:
# now we iterrate through all the test videos and produce the corresponding meta data
basePath = '/content/drive/MyDrive/Colab Notebooks/Youtube/output/csvData'
jsonPath = "/content/drive/MyDrive/Colab Notebooks/Youtube/json"
totalJson = []
for fileName in os.listdir(basePath):
  videoName = fileName.split(".")[0]
  print("procesing ", str(videoName))
  p, f = predictData(videoName, basePath)
  data = createJson(p, f, testInfo[videoName])
  totalJson += data
path = os.path.join(jsonPath, "totalJson.csv")
with open(path, "w+") as jsonOutput:
    jsonOutput.write(json.dumps(totalJson))


procesing  test_10
now processing /content/drive/MyDrive/Colab Notebooks/Youtube/output/csvData/test_10.csv
shape of x_test: 
(14469, 1, 63)
number of landmarks: 
14469
procesing  test_08
now processing /content/drive/MyDrive/Colab Notebooks/Youtube/output/csvData/test_08.csv
shape of x_test: 
(16221, 1, 63)
number of landmarks: 
16221
procesing  test_09
now processing /content/drive/MyDrive/Colab Notebooks/Youtube/output/csvData/test_09.csv
shape of x_test: 
(6719, 1, 63)
number of landmarks: 
6719
procesing  test_03
now processing /content/drive/MyDrive/Colab Notebooks/Youtube/output/csvData/test_03.csv
shape of x_test: 
(14225, 1, 63)
number of landmarks: 
14225
procesing  test_07
now processing /content/drive/MyDrive/Colab Notebooks/Youtube/output/csvData/test_07.csv
shape of x_test: 
(4852, 1, 63)
number of landmarks: 
4852
procesing  test_05
now processing /content/drive/MyDrive/Colab Notebooks/Youtube/output/csvData/test_05.csv
shape of x_test: 
(4580, 1, 63)
number of landmarks

In [58]:
with open(path, "r") as jsonInput:
    t = json.load(jsonInput)
print(len(t))

852
